<a href="https://colab.research.google.com/github/GarimaSharda/Kaggle-Knowledge-Project-Titanic/blob/main/Kaggle_Titanic_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Upload local CSV file from your computer
from google.colab import files
uploaded = files.upload()

# Step 2: Load it into pandas
import pandas as pd
train = pd.read_csv('train.csv')

# Step 3: Check the data
train.head()


Saving train.csv to train.csv


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# Upload gender_submission.csv from your computer
uploaded = files.upload()

# Read into pandas
gender_submission = pd.read_csv('gender_submission.csv')

# Display first few rows
gender_submission.head()


Saving gender_submission.csv to gender_submission.csv


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [ ]:
# Upload file manually from your computer
uploaded = files.upload()

# Load CSV file into a DataFrame
test = pd.read_csv('test.csv')

# Display the first few rows
test.head()

Saving test.csv to test.csv


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
#Get the number of entries in each file
train_len = len(train)
test_len = len(test)

print("Number of entries in train.csv:", train_len)
print("Number of entries in test.csv:", test_len)

Number of entries in train.csv: 891
Number of entries in test.csv: 418


In [ ]:
#check null value
print(train.isnull().sum())
print(test.isnull().sum())


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [ ]:
test['Ticket_Number'] = test['Ticket'].astype(str).str.extract(r'(\d+)$', expand=False)
test[['Ticket', 'Ticket_Number']].head(30)

,Ticket,Ticket_Number
0,330911,330911
1,363272,363272
2,240276,240276
3,315154,315154
4,3101298,3101298
5,7538,7538
6,330972,330972
7,248738,248738
8,2657,2657
9,A/4 48871,48871


In [ ]:

# -------------------------
# 1. AGE (missing in train + test)
# -------------------------
# Strategy: median Age by Sex and Pclass (a common, reasonable choice)

for df in [train, test]:
    df["Age"] = df["Age"].astype(float)
    df["Age"] = df.groupby(["Sex", "Pclass"])["Age"].transform(
        lambda x: x.fillna(x.median())
    )

In [ ]:
# -------------------------
# 2. EMBARKED (missing only in train)
# -------------------------
# Strategy: fill with mode (most frequent port)

embarked_mode = train["Embarked"].mode()[0]
train["Embarked"] = train["Embarked"].fillna(embarked_mode)


In [ ]:
# -------------------------
# 3. FARE (missing only in test)
# -------------------------
# Strategy: median Fare by Pclass

test["Fare"] = test["Fare"].astype(float)
test["Fare"] = test.groupby("Pclass")["Fare"].transform(
    lambda x: x.fillna(x.median())
)

In [ ]:
# -------------------------
# 4. CABIN (mostly missing in both)
# -------------------------
# Strategy: create a binary flag and drop the original Cabin

for df in [train, test]:
    df["HasCabin"] = df["Cabin"].notna().astype(int)
    df.drop(columns=["Cabin"], inplace=True)


In [ ]:
# -------------------------
# 5. Sanity check: no missing values remain
# -------------------------
print("Missing values in train:\n", train.isna().sum())
print("\nMissing values in test:\n", test.isna().sum())

Missing values in train:
 PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
HasCabin       0
dtype: int64

Missing values in test:
 PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         0
Ticket_Number    0
HasCabin         0
dtype: int64


In [ ]:
#create additional features
for df in [train, test]:
    # Family size and isolation
    df["FamilySize"] = df["SibSp"] + df["Parch"] + 1
    df["IsAlone"] = (df["FamilySize"] == 1).astype(int)

    # Extract title from Name
    df["Title"] = df["Name"].str.extract(r',\s*([^\.]+)\.', expand=False)
    df["Title"] = df["Title"].replace(
        ["Lady", "Countess", "Capt", "Col", "Don", "Dr", "Major", "Rev",
         "Sir", "Jonkheer", "Dona"],
        "Rare"
    )
    df["Title"] = df["Title"].replace(
        {"Mlle": "Miss", "Ms": "Miss", "Mme": "Mrs"}
    )


In [ ]:


# Map Sex
for df in [train, test]:
    df["Sex"] = df["Sex"].map({"male": 0, "female": 1}).astype(int)

# One‑hot encode Embarked and Title
train = pd.get_dummies(train, columns=["Embarked", "Title"], drop_first=True)
test  = pd.get_dummies(test,  columns=["Embarked", "Title"], drop_first=True)

# Align columns (ensure same dummies in train and test)
train, test = train.align(test, join="left", axis=1, fill_value=0)


In [ ]:
for dataset in [train, test]:
    dataset['Age'].fillna(dataset['Age'].median(), inplace=True)
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace=True)
    dataset.drop('Cabin', axis=1, inplace=True)

test['Fare'].fillna(test['Fare'].median(), inplace=True)


/tmp/ipython-input-2008241137.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset['Age'].fillna(dataset['Age'].median(), inplace=True)
/tmp/ipython-input-2008241137.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr